In [29]:
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import plotly.graph_objects as go

# Загрузка данных
file_path = 'Данные вариант 1.xlsx'
xls = pd.ExcelFile(file_path)
products = pd.read_excel(xls, sheet_name='Products')
dynamic  = pd.read_excel(xls, sheet_name='Dynamic')

# Названия колонок
rev_nov     = 'Выручка (Ноябрь)'
rev_dec     = 'Выручка (Декабрь)'
sales_nov   = 'Продажи (Ноябрь)'
sales_dec   = 'Продажи (Декабрь)'
price_nov   = 'Цена (Ноябрь)'
price_dec   = 'Цена (Декабрь)'
buy_nov     = 'Средний % выкупа (Ноябрь)'
buy_dec     = 'Средний % выкупа (Декабрь)'

# Считаем %-дельты
products['Delta_Revenue_%'] = (products[rev_dec]   / products[rev_nov] - 1) * 100
products['Delta_Price_%']   = (products[price_dec] / products[price_nov] - 1) * 100

# Преобразуем дату в листе Dynamic
dynamic['Месяц'] = pd.to_datetime(dynamic['Месяц'], format='%Y-%m')

# Инициализация Dash
app = Dash(__name__)
app.title = "Интеллектуальный финансовый дашборд"

# Диапазон для DatePickerRange
min_date = dynamic['Месяц'].min().date()
max_date = dynamic['Месяц'].max().date()

# Лэйаут
app.layout = html.Div(style={'fontFamily':'Arial, sans-serif','margin':'10px'}, children=[

    html.H1("Финансовый дашборд", style={'textAlign':'center'}),

    # Фильтры
    html.Div(style={'display':'flex','gap':'20px','flexWrap':'wrap','justifyContent':'center'}, children=[
        html.Div([
            html.Label("Бренд"),
            dcc.Dropdown(
                id='brand-dropdown',
                options=[{'label':b,'value':b} for b in sorted(products['Бренд'].unique())],
                value=sorted(products['Бренд'].unique())[0],
                clearable=False
            )
        ], style={'width':'200px'}),

        html.Div([
            html.Label("Страна"),
            dcc.Dropdown(id='country-dropdown', clearable=False)
        ], style={'width':'200px'}),

        html.Div([
            html.Label("Категория"),
            dcc.Dropdown(id='category-dropdown', clearable=False)
        ], style={'width':'200px'}),

        html.Div([
            html.Label("Период"),
            dcc.DatePickerRange(
                id='date-picker',
                start_date=min_date,
                end_date=max_date,
                min_date_allowed=min_date,
                max_date_allowed=max_date,
                display_format='YYYY-MM'
            )
        ], style={'width':'260px'}),
    ]),

    # KPI
    html.Div(id='kpi-cards',
             style={'display':'flex','justifyContent':'space-around','marginTop':'30px'}),

    # Бар-чарт
    html.Div([
        dcc.Graph(id='bar-chart', config={'scrollZoom':True})
    ], style={'marginTop':'40px'}),

    # Временной ряд
    html.Div([
        dcc.Graph(id='time-series', config={'scrollZoom':True})
    ], style={'marginTop':'40px'}),

])

# Каскадные dropdown по бренду
@app.callback(
    Output('country-dropdown','options'),
    Output('country-dropdown','value'),
    Output('category-dropdown','options'),
    Output('category-dropdown','value'),
    Input('brand-dropdown','value')
)
def update_cascades(brand):
    df = products[products['Бренд']==brand]
    countries  = sorted(df['Страна'].unique())
    categories = sorted(df['Категория'].unique())
    return (
        [{'label':c,'value':c} for c in countries], countries[0],
        [{'label':cat,'value':cat} for cat in categories], categories[0]
    )

# Обновление KPI и графиков
@app.callback(
    Output('kpi-cards','children'),
    Output('bar-chart','figure'),
    Output('time-series','figure'),
    Input('brand-dropdown','value'),
    Input('country-dropdown','value'),
    Input('category-dropdown','value'),
    Input('date-picker','start_date'),
    Input('date-picker','end_date'),
)
def update_dashboard(brand, country, category, start_date, end_date):
    # Фильтруем
    df_prod = products.query("Бренд==@brand and Страна==@country and Категория==@category")
    df_dyn  = dynamic.query("Бренд==@brand and Месяц>=@start_date and Месяц<=@end_date")

    # KPI
    rev_n   = df_prod[rev_nov].sum()
    rev_d   = df_prod[rev_dec].sum()
    rev_mom = (rev_d/rev_n - 1)*100 if rev_n else 0
    sold_d  = df_prod[sales_dec].sum()
    avg_pr  = df_prod[price_dec].mean()
    buy_rt  = df_prod[buy_dec].mean()

    kpis = [
        {"title":"Выручка (дек)","value":f"{rev_d:,.0f} ₽"},
        {"title":"Рост M/M","value":f"{rev_mom:.2f} %"},
        {"title":"Продажи (дек)","value":f"{int(sold_d):,} шт."},
        {"title":"Avg цена","value":f"{avg_pr:,.0f} ₽"},
        {"title":"% выкупа","value":f"{buy_rt:.2f} %"}
    ]
    cards=[]
    for k in kpis:
        cards.append(html.Div([
            html.H4(k["title"], style={'margin':'5px 0'}),
            html.P(k["value"], style={'fontSize':'20px','margin':'0'})
        ], style={
            'border':'1px solid #ccc','padding':'15px','borderRadius':'8px',
            'width':'150px','textAlign':'center','boxShadow':'2px 2px 5px rgba(0,0,0,0.1)'
        }))

    # Бар-чарт по SKU
    df_bar = df_prod.melt(
        id_vars=['SKU'],
        value_vars=[rev_nov, rev_dec],
        var_name='Месяц', value_name='Выручка'
    )
    fig_bar = px.bar(df_bar, x='SKU', y='Выручка', color='Месяц',
                     title=f"Выручка по SKU: {brand} / {country} / {category}",
                     template='plotly_white')
    fig_bar.update_layout(xaxis_tickangle=-45,
                         margin={'t':50,'b':150},
                         legend={'orientation':'h','y':-0.2})

    # Линейный график выручка и % выкупа
    fig_ts = go.Figure()
    fig_ts.add_trace(go.Scatter(
        x=df_dyn['Месяц'], y=df_dyn['Выручка'],
        mode='lines+markers', name='Выручка', yaxis='y1'
    ))
    fig_ts.add_trace(go.Scatter(
        x=df_dyn['Месяц'], y=df_dyn['Средний % выкупа'],
        mode='lines+markers', name='% выкупа', yaxis='y2'
    ))
    fig_ts.update_layout(
        title=f"Динамика бренда «{brand}»",
        template='plotly_white',
        xaxis=dict(title='Месяц'),
        yaxis=dict(title='Выручка', side='left'),
        yaxis2=dict(title='% выкупа', side='right', overlaying='y'),
        legend=dict(orientation='h', y=-0.2),
        margin={'t':50,'b':100}
    )

    return cards, fig_bar, fig_ts

# Запуск
if __name__ == '__main__':
    app.run(mode='inline', debug=False)